# QuickStart

We will do the assembly and merging using the data in the example_data directory in the following order:

1. Preparing sample info spreadsheet
2. Preparing inputs
3. Assembling individual samples
4. Evalulation of individual assemblies
5. Preparing for merge
6. Merging
7. Evaluation of the merged assembly

The example data is the chromosome 1 portion of the RNASeq data from the neurons in superior central nucleus raphe, medial part (CSm) and dosal nucleus raphe (DR) in Fev-Cre mouse line which labels serotoninergic neurons.  (Fev-Cre is also called ePet-Cre.) 


## Sample Info

Prepare an Excel spreadsheet or a tab separated text file which contains following fields for each sample.

Required fields:

- **name**: unique sample name
- **bwfile**: location of normalized coverage in BIGWIG format
- **sjfile**: location of splice junction table

Optional fileds:

- **sample_id**: unique sample identifying number
- **group**: sample group
- **bamfile**: BAM file location
- **sjtabfile**: location of STAR SJ.out.tab
- **aligned**: number of aligned reads



In [1]:
# This is to change logging level of jupyter notebook
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')

In [1]:
import pandas as PD
import os

si = PD.read_excel('../tests/data/sampleinfo.xlsx')
print(si[['name','bam','bigwig','sjtab','aligned']])

               name                   bam                    bigwig  \
0   Fev_DR_m70_1623   Fev_DR_m70_1623.bam   Fev_DR_m70_1623.chr1.bw   
1  Fev_CSm_m70_1624  Fev_CSm_m70_1624.bam  Fev_CSm_m70_1624.chr1.bw   
2   Fev_DR_m71_1625   Fev_DR_m71_1625.bam   Fev_DR_m71_1625.chr1.bw   
3  Fev_CSm_m71_1626  Fev_CSm_m71_1626.bam  Fev_CSm_m71_1626.chr1.bw   

                              sjtab   aligned  
0   Fev_DR_m70_1623.chr1.SJ.out.tab  24313418  
1  Fev_CSm_m70_1624.chr1.SJ.out.tab  25857770  
2   Fev_DR_m71_1625.chr1.SJ.out.tab  26260366  
3  Fev_CSm_m71_1626.chr1.SJ.out.tab  26083777  


In [2]:
BASEDIR = '../tests/data'
si['bwfile'] = [os.path.join(BASEDIR, "bigwig", x) for x in si['bigwig']]
si['sjtabfile'] =  [os.path.join(BASEDIR, "SJ", x) for x in si['sjtab']]

print(si[['name','bwfile','sjtabfile']])

               name                                         bwfile  \
0   Fev_DR_m70_1623   ../tests/data/bigwig/Fev_DR_m70_1623.chr1.bw   
1  Fev_CSm_m70_1624  ../tests/data/bigwig/Fev_CSm_m70_1624.chr1.bw   
2   Fev_DR_m71_1625   ../tests/data/bigwig/Fev_DR_m71_1625.chr1.bw   
3  Fev_CSm_m71_1626  ../tests/data/bigwig/Fev_CSm_m71_1626.chr1.bw   

                                           sjtabfile  
0   ../tests/data/SJ/Fev_DR_m70_1623.chr1.SJ.out.tab  
1  ../tests/data/SJ/Fev_CSm_m70_1624.chr1.SJ.out.tab  
2   ../tests/data/SJ/Fev_DR_m71_1625.chr1.SJ.out.tab  
3  ../tests/data/SJ/Fev_CSm_m71_1626.chr1.SJ.out.tab  


## Preparing Inputs

You can prepare your normalized coverage bigwig files and junction tables (format will be explained below) anyway you want, but here is one way you can prepare these from the outpus of STAR mapper (BAM and SJ.out.tab). 

To generate coverage bigwigs, use function [jgem.bigwig.bam2bw](modules.html#jgem.bigwig.bam2bw):

```python
from jgem import bigwig as BW
from jgem import utils as UT

chromsizes = UT.chromsizes('mm10')
for bam, bw, aligned in si[['bamfile','bwfile','aligned']].values:
    BW.bam2bw(bam,chromsizes,bw,aligned)
```

To generate junction files from SJ.out.tab, use function [jgem.utils.sjtab2sjbed](modules.html#jgem.gtfgffbed.sjtab2sjbed):


In [3]:
# create destinations
si['sjfile'] = [os.path.join(BASEDIR, "SJ", x[:-10]+'sj.bed.gz') for x in si['sjtab']]
print(si[['name','sjfile']])

               name                                            sjfile
0   Fev_DR_m70_1623   ../tests/data/SJ/Fev_DR_m70_1623.chr1.sj.bed.gz
1  Fev_CSm_m70_1624  ../tests/data/SJ/Fev_CSm_m70_1624.chr1.sj.bed.gz
2   Fev_DR_m71_1625   ../tests/data/SJ/Fev_DR_m71_1625.chr1.sj.bed.gz
3  Fev_CSm_m71_1626  ../tests/data/SJ/Fev_CSm_m71_1626.chr1.sj.bed.gz


In [4]:
%%time
# convert from SJ.out.tab to appropriate BED file
from jgem import gtfgffbed as GGB

for sjtab, sjbed, aligned in si[['sjtabfile','sjfile','aligned']].values:
    GGB.sjtab2sjbed(sjtab,sjbed,aligned)

CPU times: user 376 ms, sys: 33.1 ms, total: 409 ms
Wall time: 513 ms


In [7]:
ls  ../tests/data/SJ/*.chr1.sj.bed.gz

../tests/data/SJ/Fev_CSm_m70_1624.chr1.sj.bed.gz
../tests/data/SJ/Fev_CSm_m71_1626.chr1.sj.bed.gz
../tests/data/SJ/Fev_DR_m70_1623.chr1.sj.bed.gz
../tests/data/SJ/Fev_DR_m71_1625.chr1.sj.bed.gz


In [9]:
from jgem import utils as UT
sj = UT.read_pandas(sjbed, names=['chr','st','ed','name','ucnt','strand','mcnt'])
print(sj.head())

    chr       st       ed                 name      ucnt strand      mcnt
0  chr1  3121707  3142317   CT/AC-k0-u4-m0-o46  0.153352      -  0.000000
1  chr1  3207318  3213438  CT/AC-k0-u10-m0-o47  0.383380      -  0.000000
2  chr1  3271573  3302745   GT/AG-k0-u0-m1-o37  0.000000      +  0.038338
3  chr1  3421902  3670551  CT/AC-k1-u20-m0-o43  0.766760      -  0.000000
4  chr1  3660890  3670551   CT/AC-k0-u2-m0-o41  0.076676      -  0.000000


### SJ txt file format

As shown above, the tab separated junction file should contain:


- **chr**:  chromosome name
- **st**: start position
- **ed**: end position
- **name**: junction name
- **strand**: strand of junction
- **ucnt**: unique reads (normalized to 1M mapped reads)
- **mcnt**: non-unique (multimapping) reads (normalized)


## Assembling individual samples

In [13]:
from jgem import filenames as FN
from jgem import assembler as AS

In [14]:
%%time
# assemble all samples
for name,bwfile,sjfile in si[['name','bwfile','sjfile']].values:
    fn = FN.FileNames(name, bwfile, sjfile, outdir='../out')
    p = AS.Assembler(fn,saveintermediates=False)
    p.assemble()

CPU times: user 2min 14s, sys: 9.67 s, total: 2min 23s
Wall time: 4min 51s


In the output directory, several files are generated:

In [3]:
ls ../out/Fev_DR_m70*

../out/Fev_DR_m70_1623.assemble.params.txt.gz
../out/Fev_DR_m70_1623.assemble.stats.txt.gz
../out/Fev_DR_m70_1623.ex2.bed
../out/Fev_DR_m70_1623.ex2.bed.gz
../out/Fev_DR_m70_1623.ex2.txt.gz
../out/Fev_DR_m70_1623.findsecovth.params.txt.gz
../out/Fev_DR_m70_1623.findsecovth.pdf
../out/Fev_DR_m70_1623.genes.bed
../out/Fev_DR_m70_1623.genes.bed.gz
../out/Fev_DR_m70_1623.sj.bed
../out/Fev_DR_m70_1623.sj.bed.gz
../out/Fev_DR_m70_1623.sj.txt.gz


Main outputs are .ex.txt.gz, .sj.txt.gz, each containing exons and junctions. BED files are provide for view with browsers like IGV. Below is a segment of chromosome 1 showing genes.bed files from 4 samples. 

![IGV2](_static/WMSUAE41D6ONM36O9K5LYVIFO0T7EKLM.png)

## Evaluation of individual assemblies

![](_static/UCNYV53JQT9QG8GJQSS36GKMGSIT22C2.png)